In [44]:
from neo4j import GraphDatabase
import pandas as pd
import networkx as nx
from tqdm import tqdm
import math
from typing import Tuple, NamedTuple, Set, List
from datetime import datetime
"
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "mag"))

In [54]:
class Author(NamedTuple):
    citationCount: int
    createdDate: datetime
    displayName: str
    paperCount: int
    rank: int
    authorId: int
    normalizedName: str

def node_to_author(node) -> Author:
    return Author(
        node.get('citationCount'),
        node.get('createdDate').to_native(),
        node.get('displayName'),
        node.get('paperCount'),
        node.get('rank'),
        node.get('authorId'),
        node.get('normalizedName')
    )

def author_cites_relations(fieldOfStudyName: str) -> List[Tuple[Author, Author]]:
    AUTHOR_CITED_QUERY = """
        MATCH (p1:Paper)-[:IN_FIELD]->(parent:FieldsOfStudy{normalizedName: $fosName}) WHERE p1.citationCount > 0
        MATCH (p1:Paper)-[r:REFERENCES]->(p2:Paper)
        MATCH (p2:Paper)-[:IN_FIELD]->(parent:FieldsOfStudy{normalizedName: $fosName}) WHERE p2 <> p1 AND p2.citationCount > 0
        MATCH (p2:Paper)-[:AUTHORED_BY]->(a2:Author) MATCH (p1:Paper)-[:AUTHORED_BY]->(a1:Author) WHERE a1 <> a2
        RETURN a1, a2
    """
    with driver.session() as session:
        return session.run(AUTHOR_CITED_QUERY, fosName=fieldOfStudyName)

        results : List[Author] = [
            (node_to_author(res['a1']), node_to_author(res['a2']))
            for res in raw_results if res['a1'].get('authorId') != res['a2'].get('authorId')]
        
        return list(set(results))


In [55]:
res = author_cites_relations('abstract algebra')

In [59]:
res[1]

{'a1': <Node id=146651891 labels={'Author'} properties={'citationCount': 70, 'createdDate': neotime.Date(2017, 6, 30), 'displayName': 'Helena M. Pycior', 'paperCount': 7, 'rank': 18057, 'authorId': 2691817264, 'normalizedName': 'helena m pycior'}>,
 'a2': <Node id=112723973 labels={'Author'} properties={'citationCount': 60, 'createdDate': neotime.Date(2017, 6, 30), 'displayName': 'Elaine Koppelman', 'paperCount': 2, 'rank': 18419, 'authorId': 2644213204, 'normalizedName': 'elaine koppelman'}>}

In [53]:
len(graph.edges())

976